In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os

import yfinance as yf
from datetime import datetime
import pytz

In [2]:
# Generate portfolio.csv if it doesn't exist
if not os.path.exists('portfolio.csv'):
    portfolio_df = pd.DataFrame(columns=[
    'market_type',
    'asset_name',
    'asset_type',
    'price',
    'quantity',
    'leverage',
    'exchange_rate',
    'fx_fee',
    'invested_amount',
    'leverage_amount',
    'total_amount_krw',
    ],
    index=pd.DatetimeIndex([]))

    portfolio_df.to_csv('portfolio.csv')
else:
    portfolio_df = pd.read_csv('portfolio.csv')

In [3]:
def convert_to_et_time(time_str):
    if isinstance(time_str, str):
        kr_time = pd.to_datetime(time_str)
    
    kr_tz = pytz.timezone('Asia/Seoul')
    kr_time = kr_tz.localize(kr_time)

    et_tz = pytz.timezone('US/Eastern')
    et_time = kr_time.astimezone(et_tz)

    return et_time

def get_price_at_time(asset_name, target_time):
    # TODO: 토스에 사용되는 주식 가격과 yfinance에서도 적용되는 주식 가격 차이 해결
    stock = yf.Ticker(asset_name)

    et_time = convert_to_et_time(target_time)

    data = stock.history(
        start=et_time.date(),
        end=et_time.date() + pd.Timedelta(days=1),
        interval='1m'
    )

    data.index = data.index.tz_localize(None)
    et_time = et_time.tz_localize(None)

    closest_time = min(data.index, key=lambda x: abs(x - et_time))
    price = data.loc[closest_time, 'Close']

    return closest_time, price

# TODO: 환율 계산 함수 추가 (기준 환율 vs 적용 환율 그리고 환율 수수료 할인 등)

In [4]:
def fill_portfolio_df(
        time,
        market_type, asset_name, asset_type, quantity, leverage,
        exchange_rate, fx_fee
    ):
    assert market_type in ['US', 'KR', 'COIN']
    assert asset_type in ['equity', 'futures', 'option', 'bond']
    
    if market_type == 'US':
        _, price = get_price_at_time(asset_name, time)

    idx = pd.to_datetime(time)
    asset_type = asset_type.lower()

    invested_amount = price * quantity
    leverage_amount = invested_amount * leverage
    total_amount_krw = invested_amount * (exchange_rate + fx_fee)

    portfolio_df.loc[idx] = {
        'market_type' : market_type,
        'asset_name' : asset_name,
        'asset_type' : asset_type,
        'price' : price,
        'quantity' : quantity,
        'leverage' : leverage,
        'exchange_rate' : exchange_rate,
        'fx_fee' : fx_fee,
        'invested_amount' : invested_amount,
        'leverage_amount' : leverage_amount,
        'total_amount_krw' : total_amount_krw,
    }

    # CSV file update
    portfolio_df.to_csv('portfolio.csv')

    return portfolio_df

In [5]:
time_str = '2024-11-27 2:57:00'
market_type = 'US'
asset_name = 'IBM'
asset_type = 'equity'
quantity = 1
leverage = 1
exchange_rate = 1398.2
fx_fee = (1407.6 - exchange_rate) # 적용 환율 - 기준 환율

fill_portfolio_df(time_str, market_type, asset_name, asset_type, quantity, leverage, exchange_rate, fx_fee)

,market_type,asset_name,asset_type,price,quantity,leverage,exchange_rate,fx_fee,invested_amount,leverage_amount,total_amount_krw
2024-11-27 02:57:00,US,IBM,equity,227.537506,1,1,1398.2,9.4,227.537506,227.537506,320281.793591
